In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_url = '/kaggle/input/jane-street-market-prediction/train.csv'
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

#new 
import numba


df = pd.read_csv(df_url)

In [ ]:
#average returns and assign a positive or negative outcome
@numba.njit(fastmath = True)
def clean_set(sample_df):
    df_y = sample_df.loc[:, 'resp_1':'resp']
    outcome = []
    avg_return = df_y.mean(axis=1)
    for i in avg_return: 
        if i > 0: 
            outcome.append(1)
        else:
            outcome.append(0)
    sample_df['outcome'] = outcome
    
    #drop individual returns, weight and rows with zero weight
    samp = sample_df.drop(['resp_1','resp_2','resp_3','resp_4','resp','date','ts_id'],axis= 1)
    df1 = samp[samp['weight'] != 0]
    df1 = df1.drop(['weight'],axis = 1)
    return df1
    
df1 = clean_set(df)

In [ ]:
#create test train split within group
X = df1.drop(['outcome'],axis=1)
y = df1['outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

In [ ]:
#fill in NANs with median
train_median = X_train.median()
test_median = X_test.median()
X_train = X_train.fillna(train_median)
X_test = X_test.fillna(test_median)

#scale features between zero and one
sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

#test explanatory value of features 
# pca = PCA()
# X_train = pca.fit_transform(X_train)
# explained_variance = pca.explained_variance_ratio_
# exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

# use PCA to conoslidate feature set to ten features
pca = PCA(n_components=10)
pca.fit(X_train)
X_train = pca.transform(X_train)
pca.fit(X_test)
X_test = pca.transform(X_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:
@numba.njit(fastmath = True)
def scorer(preds, reals):
    acc_count = 0
    for i in range(len(preds)):
        if preds[i] == reals[i]:
            acc_count += 1
    acc_rate = acc_count / len(preds)
    return acc_rate

from sklearn.linear_model import LogisticRegression
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(scorer('SVC: ', preds, list(y_test)))